In [1]:
import pandas as pd
import numpy as np
import importlib
from leak_detect.base import * 

In [19]:
#import leak_detect.base
#importlib.reload(leak_detect.base) #doesnt work
#?detect_vertical_leakage# 

<module 'leak_detect.base' from '/Users/abhaypawar/Documents/projects/leak_detection/leak-detect/leak_detect/base.py'>

In [2]:
data = pd.read_csv('data/daily_adjusted_AAPL.csv')
prediction_window = 5
data['target'] = data['open'].shift(-prediction_window)


## Horizontal leakge

In [6]:
# Data creation function
def create_features(data_input):
    data = data_input.copy()
    # past 2 and 5 day return
    close_5days_before = data['close'].shift(5)
    close_2days_before = data['close'].shift(2)
    
    data['return_5day'] = (data['close'] - close_5days_before)/close_5days_before
    data['return_2day'] = (data['close'] - close_2days_before)/close_2days_before
    data['open_1day_before_leaky'] = data['open'].shift(-1)
    data['target'] = data['close']*3
    
    return data


In [ ]:
target_cols = ['target']
input_feature_cols = ['open', 'high', 'low', 'close']
output_feature_cols = ['return_5day', 'return_2day', 'open_1day_before_leaky']


In [7]:
detect_horizontal_leakage(create_features, data, target_cols, output_feature_cols, input_feature_cols)

Checking for leakage from target columns to feature columns...
By replacing target with NANs:
No horizontal leakage detected. Good to go! Yay!!


By adding imaginary component to target columns:
No horizontal leakage detected. Good to go! Yay!!


---------------------------------------------------------------------------
Checking for leakage from input feature columns to target columns...
By replacing input feature columns with NANs:
Oops horizontal leakage detected!! 
List of columns and their respective number of rows with leaky data:
target : 5033


By adding imaginary component to input feature columns:
Oops horizontal leakage detected!! 
List of columns and their respective number of rows with leaky data:
target : 5033




True

## Vertical leakge

In [14]:
# Data creation function
def create_features(data_input):
    data = data_input.copy()
    # past 2 and 5 day return
    close_5days_before = data['close'].shift(-4)
    close_2days_before = data['close'].shift(-22)
    
    data['return_5day'] = (data['close'] - close_5days_before)/close_5days_before
    data['return_2day'] = (data['close'] - close_2days_before)/close_2days_before
    data['open_1day_before_leaky'] = data['open'].shift(1)
    data['target'] = data['close'].shift(1)*3
    return data


In [15]:
input_feature_cols = ['open', 'high', 'low', 'close', 'target']
output_feature_cols = ['return_5day', 'return_2day', 'open_1day_before_leaky', 'target']
check_row_number = -1


In [18]:
detect_vertical_leakage(create_features, data, input_feature_cols, output_feature_cols, only_nan=False, 
                       direction='upward', check_row_number=-1)


Checking for vertical leakage in upward direction...
By replacing 'input_feature_cols' after row number 2516 with NANs:
Oops vertical leakage detected!!
List of columns and number of previous rows into which data is leaking from a row:
return_5day : 4
return_2day : 22


By adding imaginary component to 'input_feature_cols' after row number 2516:
Oops vertical leakage detected!!
List of columns and number of previous rows into which data is leaking from a row:
return_5day : 4
return_2day : 22




True